# Approach 1 :- MASKING

In [4]:
import cv2

# load images
image1 = cv2.imread("input1.png")
image2 = cv2.imread("input2.png")

# compute difference
difference = cv2.subtract(image1, image2)

# color the mask red
Conv_hsv_Gray = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(Conv_hsv_Gray, 0, 255,cv2.THRESH_BINARY_INV |cv2.THRESH_OTSU)
difference[mask != 255] = [0, 0, 255]

# add the red mask to the images to make the differences obvious
image1[mask != 255] = [0, 0, 255]
image2[mask != 255] = [0, 0, 255]

# store images
cv2.imwrite('diffOverImage1.png', image1)
cv2.imwrite('diffOverImage2.png', image1)
cv2.imwrite('diff.png', difference)

True

# Approach 2:- Structural Similarity Index (SSIM)
#### Disadvantage:- Both file must be same size.

In [14]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

# Load images
before = cv2.imread("input1.png")
after = cv2.imread("input2.png")

# Convert images to grayscale
before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

# Compute SSIM between the two images
(score, diff) = structural_similarity(before_gray, after_gray, full=True)
print("Image Similarity: {:.4f}%".format(score * 100))

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")
diff_box = cv2.merge([diff, diff, diff])

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(before.shape, dtype='uint8')
filled_after = after.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(diff_box, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (255,255,255), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

cv2.imshow('before', before)
cv2.imshow('after', after)
cv2.imshow('diff', diff)
cv2.imshow('diff_box', diff_box)
cv2.imshow('mask', mask)
cv2.imshow('filled after', filled_after)
cv2.waitKey()

Image Similarity: 93.8810%


-1

# Approach3:- cv2.absdiff
### Disadvantage:- Not calculate similarity Score

In [2]:
import cv2

# Load images as grayscale
image1 = cv2.imread("input1.png", 0)
image2 = cv2.imread("input2.png", 0)

# Calculate the per-element absolute difference between 
# two arrays or between an array and a scalar
diff = 255 - cv2.absdiff(image1, image2)

cv2.imshow('diff', diff)
cv2.waitKey()

-1

In [6]:
import cv2
import numpy as np

# Load the two input images
image1 = cv2.imread("input1.png")
image2 = cv2.imread("input2.png")

# Convert images to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Compute absolute difference between the two images
diff = cv2.absdiff(gray1, gray2)

# Threshold the difference image
_, threshold = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

# Find contours of the thresholded image
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around the contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Calculate similarity score (percentage of pixels that are different)
total_pixels = np.prod(gray1.shape)
different_pixels = np.count_nonzero(diff)
similarity_score = (1 - (different_pixels / total_pixels)) * 100

# Display the result
cv2.imshow('Difference Image', diff)
cv2.imshow('Threshold Image', threshold)
cv2.imshow('Bounding Boxes', image1)
print(f"Similarity Score: {similarity_score:.2f}%")

cv2.waitKey(0)
cv2.destroyAllWindows()


Similarity Score: 94.76%


In [7]:
import cv2

def detect_difference(image1, image2):
  # Preprocessing
  image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
  image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

  # Difference Image
  diff_image = cv2.absdiff(image1_gray, image2_gray)

  # Thresholding
  thresh = cv2.threshold(diff_image, 20, 255, cv2.THRESH_BINARY)[1]

  # Find Contours and Draw Bounding Boxes
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(image1, (x, y), (x+w, y+h), (0, 0, 255), 2)

  return image1

# Calculate Similarity Score (Mean Squared Error)
def mse(image1, image2):
  diff = image1.astype("float") - image2.astype("float")
  squared_diff = diff ** 2
  mse = squared_diff.mean()
  return mse

# Example Usage
image1 = cv2.imread("input1.png")
image2 = cv2.imread("input2.png")
image_with_boxes = detect_difference(image1, image2)

# Print Similarity Score
similarity_score = mse(image1, image2)
print(f"Similarity Score (MSE): {similarity_score}")

# Display Image with Bounding Boxes
cv2.imshow("Image with Bounding Boxes", image_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()

Similarity Score (MSE): 887.4431480995539


# Approach3:- Animated using flicker_cmp

In [10]:
# flicker_cmp -o result.gif -r x400 a.jpg b.jpg
from PIL import Image

# Create a list of frames of the animation
frames = []
frames.append(Image.open('input1.png'))
frames.append(Image.open('input2.png'))

# Save result
frames[0].save('anim.gif', save_all=True, append_images=frames[1:], duration=100, loop=0)

In [10]:

#pixel wise comparision
import cv2

# Load the images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Compute the absolute difference between the images
difference = cv2.absdiff(image1, image2)

# Convert the difference image to grayscale
gray = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)

# Threshold the grayscale image to get a binary mask of the differences
_, threshold = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)

# Find contours in the thresholded image
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around the contours
for contour in contours:
    (x, y, w, h) = cv2.boundingRect(contour)
    cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the images
cv2.imshow('Original Image', image1)
cv2.imshow('Difference Image', difference)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [13]:
#histogram diffrence
import cv2
import numpy as np

# Load the images
image1 = cv2.imread('input1.png')
image2 = cv2.imread('input2.png')

# Calculate the histograms
hist1 = cv2.calcHist([image1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist2 = cv2.calcHist([image2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])

# Calculate the Chi-Squared distance
difference = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR)

print(f"Histogram Difference: {difference}")


Histogram Difference: 18066.291655547288


In [12]:
#feature Based matching
import cv2
import numpy as np

# Load the images
image1 = cv2.imread('input1.png', 0)
image2 = cv2.imread('input2.png', 0)

# Initialize the ORB feature detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(image1, None)
kp2, des2 = orb.detectAndCompute(image2, None)

# Create a Brute Force Matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(des1, des2)

# Sort them in the order of their distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw the first 10 matches
result = cv2.drawMatches(image1, kp1, image2, kp2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
cv2.imshow('Matches', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [6]:
pwd


'f:\\picdiffbox\\Resources'